In [ ]:
import requests, json
from lxml import html
from bs4 import BeautifulSoup
import pandas as pd
import pickle

## 登入
* 登入後重複用 `session_request` 去爬其他資料

In [ ]:
EMAIL = 'tony71318@yahoo.com.tw'
PASSWORD = 'Bot12345'

LOGIN_URL = 'https://www.bundesliga.com/en/bundesliga/login/'
api_key = '3_tQWqB95PCJiqCeljbI7N3oOEiX_q4-wobViZ6EYqtb6vYsZDYQqgPT1Z4O6btnKh'

In [ ]:
headers = {
    'Cookie': 'gmid=J8dn6yAqHjxSZ8kTmbjmVno6Dn3eBcM2GCcc9wR95jI=; hasGmid=ver2; apiDomain_3_dmGIVZ011qA426N4BdhbPVrAqPj8Ve_z6Y1OIC9wOXz2bvKj37HhXXqcaKsESJIE=eu1.gigya.com; ucid=tIzK1ZQCy7sH8VQXbT7itg',
    'Referer': 'https://cdns.eu1.gigya.com/gs/webSdk/Api.aspx?apiKey=3_tQWqB95PCJiqCeljbI7N3oOEiX_q4-wobViZ6EYqtb6vYsZDYQqgPT1Z4O6btnKh&version=latest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                   'AppleWebKit/537.36 (KHTML, like Gecko)'
                   'Chrome/69.0.3497.92 Safari/537.36',
}

In [ ]:
payload = {
    'loginID': EMAIL,
    'password': PASSWORD,
    'sessionExpiration': '0',
    'targetEnv': 'jssdk',
    'include': 'profile,data,emails,subscriptions,preferences,',
    'includeUserInfo': 'true',
    'loginMode': 'standard',
    'lang': 'en',
    'APIKey': api_key,
    'source': 'showScreenSet',
    'sdk': 'js_latest',
    'authMode': 'cookie',
    'pageURL': 'https://www.bundesliga.com/en/bundesliga/login/',
    'format': 'jsonp',
    'callback': 'gigya.callback',
    'context': 'R1396585807',
    'utf8': '&#x2713'
}

In [ ]:
session_requests = requests.session()
result = session_requests.post(LOGIN_URL, data = payload, headers = headers)
result

## 得分排行榜
* 回傳: json格式
    * Metadata
    * Players
    * Rankings
    * Teams
    
**TODO: 網頁表格 是從這個json組出來的 交給你想辦法(DONE)**

In [ ]:
headers = {
    'Referer': 'https://www.bundesliga.com/en/stats/bundesliga/current-season/all-matchdays/player-stats/goals',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)' 
    'AppleWebKit/537.36 (KHTML, like Gecko)'
    'Chrome/69.0.3497.92 Safari/537.36',
    'x-api-key': 'HEEyMDwtUkR8FUbSj8bx8jqy'
}

In [ ]:
api_url = 'https://xthokrkm7l.execute-api.eu-central-1.amazonaws.com/Prod/ranking?feedType=Goals&timeScope=Season&dataScope=Competition&competitionId=DFL-COM-000001&seasonId=DFL-SEA-0001K2&matchDayId=DFL-DAY-004BPN&rankingType=GOALS&rankingScope=PLAYER'
url = session_requests.get(api_url, headers = headers) 
score_json = json.loads(url.text)
score_json

In [ ]:
# 將三個資料存成df(暫時不知道可以怎麼用 先不存)
players = pd.DataFrame(score_json['Players'])
rankings = pd.DataFrame(score_json['Ranking'])
teams = pd.DataFrame(score_json['Teams'])

## Matchday 1-34 2017/18
* 整理成df格式
* pickle檔案為保留型態的檔案(之後分析好用)

** 待解決 **
* 找出有用的columns(不急)

In [ ]:
# 抓matchday1~34 2017/18
matchday = {}
for i in range(34):
    api_url = 'https://www.bundesliga.com/fsSolrSearch/search?wt=json&rows=150&sort=kickoffTimeMS_l%20asc&q=indexName%3ASyncMatches%20AND%20competitionID_tlc%3ADFL-COM-000001%20AND%20seasonID_tlc%3A2017%2F2018%20AND%20matchday_l%3A'+str(i+1)
    url = session_requests.get(api_url, headers = headers) 
    matchday[i] = json.loads(url.text)

In [ ]:
df_matchday = pd.DataFrame(matchday[0]['response']['docs'])
for i in range(1,34):
    df_matchday = pd.concat([df_matchday,pd.DataFrame(matchday[i]['response']['docs'])])

In [ ]:
# 把資料存起來
with open('Data/matchday.pickle', 'wb') as handle:
    pickle.dump(df_matchday, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# 開啟方式如下
# with open('Data/matchday.pickle', 'rb') as handle:
#     df_matchday = pickle.load(handle)

## TODO
* 希望可以用程式直接連button作後續的處理
* 連結單場比賽獲得單場各項數據(連不到可以用的html)

In [ ]:
# 找不到合適的api_url
api_url = 'https://www.bundesliga.com/data/feed/51/2017/player_stats/player_stats_168428.xml?cb=380141'
url = session_requests.get(api_url, headers = headers) 
# matchday = json.loads(url.text)
# matchday
soup = BeautifulSoup(url.content,'xml')
soup